### Intercambio sesgo-varianza (bias-variance tradeoff)

#### Sesgo

Cuando generamos conclusiones a partir de un modelo ajustado a cierto conjunto de datos podemos encontrarnos en una de dos situaciones. La primera involucra que tratemos de medir o evaluar qué tan bien se ajustan los datos a un modelo teórico previo, este "ajuste de los datos" concretamente se vuelve en qué tan bien los parámetros muestrales que estemos proponiendo se ajustan al parámetro poblacional real. Esta desviación es conocida como sesgo y su definición es la siguiente:

$$Sesgo~de~un~estimador~\hat\theta_n: E[\hat\theta_n]-\theta$$

en ésta, se tiene que estamos tratando de estimar el valor de un parámetro poblacional $\theta$ a partir de la esperanza de varios parámetros muestrales $\hat\theta$ correspondientes cada una a una muestra particular de tamaño $n$. Esta cantidad surge dentro del enfoque de la estadística inferencial clásica o paramétrica, en donde se tiene una distribución teórica de referencia para los datos y se busca evaluar qué tanto los parámetros estimados se desvían de "los que deberían ser" los valores de los parámetros reales.

En un caso ideal lo que pretenderíamos es que el estimador que estamos utilizando tenga un sesgo pequeño o tendiente a cero. Ya que si podemos probar esto teóricamente entonces podemos confiar en que si nuestros datos sí se distribuyen de la manera en que estamos suponiendo encontraremos un buen valor para el estimador del valor real del parámetro poblacional.

#### Riesgo (riesgo cuadrático)

Sin embargo, en la práctica el elegir un estimador insesgado (*i.e.* con sesgo 0) no suele garantizar una buena estimación. Por ejemplo, podríamos elegir un valor constante como estimador. Éste ejercicio aunque trivial acarrea como consecuencia que nuestro estimador tenga una varianza de cero, sin embargo su sesgo es tanto como la diferencia entre su valor y el parámetro real $\theta$. Por otra parte podemos elegir a la esperanza del conjunto de datos como estimador y aunque su sesgo tiende a cero, si el tamaño de muestra no es "lo suficientemente grande" o el muestreo no fue "representativo", nos arriesgamos a tener valores de varianza grandes. Es entonces que podemos considerar el riesgo o riesgo cuadrático de un estimador $\hat\theta_n$, como una medida que nos permita cuantificar ambos escenarios.

El riesgo cuadrático se define de la siguiente manera:

$$Riesgo^2~de~un~estimador~\hat\theta_n: E[(\hat\theta_n-\theta)^2]$$

Haciendo un poco de álgebra:

$$Riesgo^2=E[(\hat\theta_n-\theta)^2]=E[\hat\theta_n^2]+\theta^2-2E[\hat\theta_n]\theta$$ 

$$Sesgo^2=(E[\hat\theta_n]-\theta)^2=E[\hat\theta_n]^2+\theta^2-2E[\hat\theta_n]\theta$$

$$Varianza=E[\hat\theta_n^2]-E[\hat\theta_n]^2$$

$$Riesgo^2=Sesgo^2+Varianza$$

**OBSERVACIÓN:** El  $Riesgo^2$  es equivalente a definir el $Error~Cuadrático~Promedio$ $(MSE)$ de un estimador $\hat\theta_n$.

Los posibles escenarios pueden ser:

![](bias_variance.jpeg)


In [ ]:
import pandas as pd
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from mlxtend.evaluate import bias_variance_decomp
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
dataframe = read_csv(url, header=None)
dataframe

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


Attribute Information:

    1. CRIM      per capita crime rate by town
    2. ZN        proportion of residential land zoned for lots over 
                 25,000 sq.ft.
    3. INDUS     proportion of non-retail business acres per town
    4. CHAS      Charles River dummy variable (= 1 if tract bounds 
                 river; 0 otherwise)
    5. NOX       nitric oxides concentration (parts per 10 million)
    6. RM        average number of rooms per dwelling
    7. AGE       proportion of owner-occupied units built prior to 1940
    8. DIS       weighted distances to five Boston employment centres
    9. RAD       index of accessibility to radial highways
    10. TAX      full-value property-tax rate per $10,000
    11. PTRATIO  pupil-teacher ratio by town
    12. B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks 
                 by town
    13. LSTAT    % lower status of the population
    14. MEDV     Median value of owner-occupied homes in $1000's

In [ ]:
### Adjusting linear model
# separate into inputs and outputs
data = dataframe.values
X, y = data[:, :-1], data[:, -1]
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# define the model
model = LinearRegression()
# estimate bias and variance
mse, bias, var = bias_variance_decomp(model, X_train, y_train, X_test, y_test, loss='mse', num_rounds=200, random_seed=1)
# summarize results
print('MSE: %.3f' % mse)
print('Bias: %.3f' % bias)
print('Variance: %.3f' % var)
bias/var

MSE: 22.418
Bias: 20.744
Variance: 1.674


12.39232538754308

In [ ]:
# Adjusting linear model over scaled data
# scaling data
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(dataframe)
data = data_scaled
# separate into inputs and outputs
X, y = data[:, :-1], data[:, -1]
# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# define the model
model = LinearRegression()
# estimate bias and variance
mse, bias, var = bias_variance_decomp(model, X_train, y_train, X_test, y_test, loss='mse', num_rounds=200, random_seed=1)
# summarize results
print('MSE: %.3f' % mse)
print('Bias: %.3f' % bias)
print('Variance: %.3f' % var)
bias/var

MSE: 0.011
Bias: 0.010
Variance: 0.001


12.392325387543028

In [ ]:
# Adjusting k nearest neighbors
rmse_val = [] #to store rmse values for different k
for K in range(20):
    K = K+1
    model = neighbors.KNeighborsRegressor(n_neighbors = K)

    model.fit(X_train, y_train)  #fit the model
    pred=model.predict(X_test) #make prediction on test set
    error = sqrt(mean_squared_error(y_test,pred)) #calculate rmse
    rmse_val.append(error) #store rmse values
    print('RMSE value for k= ' , K , 'is:', error)

RMSE value for k=  1 is: 0.12329595496719975
RMSE value for k=  2 is: 0.08990828475358655
RMSE value for k=  3 is: 0.09370807343045032
RMSE value for k=  4 is: 0.09819941375277315
RMSE value for k=  5 is: 0.10066900045616058
RMSE value for k=  6 is: 0.1059645905547054
RMSE value for k=  7 is: 0.10701778052847595
RMSE value for k=  8 is: 0.1096279187456035
RMSE value for k=  9 is: 0.1129961695383228
RMSE value for k=  10 is: 0.11610742369044125
RMSE value for k=  11 is: 0.11838902182196599
RMSE value for k=  12 is: 0.1176068493024546
RMSE value for k=  13 is: 0.11675037879516546
RMSE value for k=  14 is: 0.11695805616580658
RMSE value for k=  15 is: 0.1188447251757602
RMSE value for k=  16 is: 0.11939981507749975
RMSE value for k=  17 is: 0.12172056535594623
RMSE value for k=  18 is: 0.12384003279010838
RMSE value for k=  19 is: 0.12517898581470946
RMSE value for k=  20 is: 0.1266423976703934


In [ ]:
# small values, such as k=1, result in a low bias and a high variance, 
# whereas large k values, such as k=21, result in a high bias and a low variance.
model = neighbors.KNeighborsRegressor(n_neighbors = 2)
# estimate bias and variance
mse, bias, var = bias_variance_decomp(model, X_train, y_train, X_test, y_test, loss='mse', num_rounds=200, random_seed=1)
# summarize results
print('MSE: %.3f' % mse)
print('Bias: %.3f' % bias)
print('Variance: %.3f' % var)
bias/var

MSE: 0.013
Bias: 0.008
Variance: 0.005


1.487357868553737

### ¿Qué elegir o qué hacer?

Lo que nosotros deseamos idealmente son modelos con bajo sesgo y baja varianza, ya que esto implicará un menor riesgo cuadrático (error cuadrático promedio). En realidad esto describe de manera bastante concisa la meta del aprendizaje de máquina aplicado; sin embargo, la realidad es que suele presentarse un *tradeoff* entre ambas medidas.

Un modelo con alto sesgo hace supuestos fuertes sobre la función subyaciente (desconocida) que funge como mapeo entre los predictores y las variables de respuesta del dataset en cuestión, tal es el caso de los modelos lineales. Por otra parte un modelo con alta varianza es un modelo que depende fuertemente de la naturaleza específica del conjunto de entrenamiento utilizado para entrenar al modelo, tal es el caso de los árboles de decisión sin poda.

En otras palabras, modelos "simples" como la regresión lineal o logística suelen presentar un alto sesgo y una baja varianza. Es decir, podemos utilizar distintos conjuntos de entrenamiento y el nivel de ajuste del modelo seguirá siendo más o menos el mismo. 

Por otra parte, modelos "más complejos", como los árboles de decisión, suelen tener un sesgo bajo y una varianza alta, lo cual puede constatarse al modificar el conjunto de entrenamiento y comparar la calidad del ajuste logrado en cada escenario.

Existen también modelos que pueden presentar un sesgo bajo y una alta varianza bajo ciertos valores para los parámetros de inicialización y un alto sesgo y baja varianza para otros. Ejemplo de esto es KNN, ya que si elegimos una valor de $k=1$ el sesgo es prácticamente nulo mientras que la varianza de las predicciones aumenta, y cuando elegimos $k=21$ esto resulta en un alto sesgo y una baja varianza.

Por lo tanto, una regla de dedo para decidir entre aumentar o disminuir el sesgo o la varianza de un modelo, siempre y cuando contemos con parámetros modificables que afecten esto, será que utilicemos como criterio de comparación el efecto conjunto que tienen ambas medidas sobre el valor del riesgo cuadrático del estimador (error cuadrático promedio), eligiendo aquella combinación de sesgo-varianza que minimice esta medida.